O serviço de vendas de carros usados Rusty Bargain está desenvolvendo um aplicativo para atrair novos clientes. Nesse aplicativo, você pode descobrir rapidamente o valor de mercado do seu carro. Você tem acesso a dados históricos: especificações técnicas, versões de acabamento e preços. Você precisa construir o modelo para determinar o valor. 

Rusty Bargain está interessado em:

- a qualidade da predição;
- a velocidade da predição;
- o tempo necessário para o treinamento

## Preparação de Dados

In [1]:
pip install scikit-learn --upgrade --user

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Pacotes para tratar os dados
import pandas as pd
import numpy as np

# Pacotes para Preparar os dados para treinamento
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

# Pacotes para treinamento de modelos
from catboost import CatBoostRegressor
import lightgbm as lgb
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# Pacotes para métricas
from sklearn.metrics import mean_squared_error
import time

# Pacotes auxiliares
import math

In [3]:
df = pd.read_csv('/datasets/car_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [4]:
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17


In [5]:
df.describe()

,Price,RegistrationYear,Power,Mileage,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


Analisando a descrição dos dados temos as seguintes conclusões:<br>
1) Existem preços com valor 0. Essas linhas deverão ser desconsideradas. <br>
2) Existem Automóveis que foram registrados no mínimo no ano 1000 e no máximo no ano 9999, o que não é possível visto que estamos em 2024 e o carro foi inventado em 1885. <br>
3) Veículos que possuem power com valor 0 também não existem. <br>
4) Registro que aconteceram no mês 0 também devem ser desconsiderados. <br>
5) A coluna NumberOfPictures é composta inteiramente por valores 0, portanto será desconsiderada.

In [6]:
# Aplicando as mudanças mencionadas
df = df[df['Price'] != 0] # 1 observação
df = df[(df['RegistrationYear'] > 1884) & (df['RegistrationYear'] < 2025)] # 2 observação
df = df[df['Power'] != 0] # 3 observação
df = df[df['RegistrationMonth'] != 0] # 4 observação
df = df.drop('NumberOfPictures', axis=1) # 5 observação
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 287100 entries, 1 to 354368
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        287100 non-null  object
 1   Price              287100 non-null  int64 
 2   VehicleType        271249 non-null  object
 3   RegistrationYear   287100 non-null  int64 
 4   Gearbox            282887 non-null  object
 5   Power              287100 non-null  int64 
 6   Model              277112 non-null  object
 7   Mileage            287100 non-null  int64 
 8   RegistrationMonth  287100 non-null  int64 
 9   FuelType           272434 non-null  object
 10  Brand              287100 non-null  object
 11  NotRepaired        249904 non-null  object
 12  DateCreated        287100 non-null  object
 13  PostalCode         287100 non-null  int64 
 14  LastSeen           287100 non-null  object
dtypes: int64(6), object(9)
memory usage: 35.0+ MB


In [7]:
# Verificando a existência de valores duplicados
df.duplicated().sum()

253

In [8]:
# Excluir
df.drop_duplicates(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 286847 entries, 1 to 354368
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        286847 non-null  object
 1   Price              286847 non-null  int64 
 2   VehicleType        270999 non-null  object
 3   RegistrationYear   286847 non-null  int64 
 4   Gearbox            282634 non-null  object
 5   Power              286847 non-null  int64 
 6   Model              276860 non-null  object
 7   Mileage            286847 non-null  int64 
 8   RegistrationMonth  286847 non-null  int64 
 9   FuelType           272182 non-null  object
 10  Brand              286847 non-null  object
 11  NotRepaired        249656 non-null  object
 12  DateCreated        286847 non-null  object
 13  PostalCode         286847 non-null  int64 
 14  LastSeen           286847 non-null  object
dtypes: int64(6), object(9)
memory usage: 35.0+ MB


In [9]:
# Preencher NotRepaired que estão nulos com unknown
df['NotRepaired'].fillna('unknown', inplace=True)

In [10]:
# Remover os outros valores nulos
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253566 entries, 2 to 354368
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        253566 non-null  object
 1   Price              253566 non-null  int64 
 2   VehicleType        253566 non-null  object
 3   RegistrationYear   253566 non-null  int64 
 4   Gearbox            253566 non-null  object
 5   Power              253566 non-null  int64 
 6   Model              253566 non-null  object
 7   Mileage            253566 non-null  int64 
 8   RegistrationMonth  253566 non-null  int64 
 9   FuelType           253566 non-null  object
 10  Brand              253566 non-null  object
 11  NotRepaired        253566 non-null  object
 12  DateCreated        253566 non-null  object
 13  PostalCode         253566 non-null  int64 
 14  LastSeen           253566 non-null  object
dtypes: int64(6), object(9)
memory usage: 31.0+ MB


## Treinamento do modelo

In [11]:
metrics = {
    'training_time(s)': [],
    'prediction_time(s)': [],
    'RMSE':[],
    'tuning_time(s)': []
}

In [12]:
# Conjunto das colunas numéricas
numerical_features = df.describe().columns.drop('Price')

# Conjunto das colunas categóricas
categoric_features = df.columns.difference(numerical_features).drop('Price')

# Removendo as colunas de datas que não agregam a predição do preço
categoric_features = categoric_features.drop(['DateCrawled', 'DateCreated', 'LastSeen'])

In [13]:
# Copiando o Dataframe em duas instâncias, um para trabalhar nos modelos que 
# requerem preprocessamento na características categóricas e outro no CatBoost e LightGBM

# Para GBM
df_gbm = df.copy()
df_gbm = df_gbm.drop(['DateCrawled', 'DateCreated', 'LastSeen'], axis=1)
# Atribuindo catacterísticas e alvo
X_gbm = df_gbm.drop(['Price'], axis=1)
y_gbm = df_gbm['Price']

X_gbm_train, X_gbm_test, y_gbm_train, y_gbm_test = train_test_split(X_gbm, y_gbm, test_size = 0.25, random_state=12345)

# Para Encoding
df_enc = df.copy()
df_enc = df_enc.drop(['DateCrawled', 'DateCreated', 'LastSeen'], axis=1)

In [14]:
def hyper_tuning(model, params, X_train, y_train, cat_features=[]):
    grid = GridSearchCV(estimator=model, param_grid=params, cv=2, n_jobs=-1, verbose=0, scoring='neg_root_mean_squared_error')
    if len(cat_features) == 0:
        start_time = time.time()
        grid.fit(X_train, y_train)
        tuning_time = time.time() - start_time
    else:
        start_time = time.time()
        grid.fit(X_gbm_train, y_gbm_train, cat_features=list(cat_features))
        tuning_time = time.time() - start_time
        
    return tuning_time, grid.best_params_


### One Hot Encoding

In [15]:
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore').set_output(transform='pandas')
ohe_transform = ohe.fit_transform(df_enc[categoric_features])
df_enc = pd.concat([df_enc, ohe_transform], axis=1).drop(columns=categoric_features)
df_enc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253566 entries, 2 to 354368
Columns: 314 entries, Price to VehicleType_wagon
dtypes: float64(308), int64(6)
memory usage: 609.4 MB


In [16]:
# Após o OHE podemos definir nossas características
X_enc = df_enc.drop(['Price'], axis=1)
y_enc = df_enc['Price']

X_enc_train, X_enc_test, y_enc_train, y_enc_test = train_test_split(X_enc, y_enc, test_size = 0.25, random_state=12345)

### LinearRegression

In [17]:
lr_model = LinearRegression()
start_time = time.time()
lr_model.fit(X_enc_train, y_enc_train)
training_time = time.time() - start_time
metrics['training_time(s)'].append(training_time)
metrics['tuning_time(s)'].append('N/A')

In [18]:
start_time = time.time()
lr_predictions = lr_model.predict(X_enc_test)
prediction_time = time.time() - start_time
metrics['prediction_time(s)'].append(prediction_time)

In [19]:
lr_rmse = math.sqrt(mean_squared_error(y_enc_test, lr_predictions))
metrics['RMSE'].append(lr_rmse)
print(f'REQM: {lr_rmse:.2f}')

REQM: 2656.54


### Random Forest

In [20]:
# Fazer Tuning de hiperparametros
rf_model = RandomForestRegressor()
rf_parameters = {
    'n_estimators': [50, 100, 150],
    'max_depth': [2, 3, 4],
    'max_features': ['sqrt', 'log2'],
    'random_state': [12345]
}

tuning_time, rf_best_params = hyper_tuning(rf_model, rf_parameters, X_enc_train, y_enc_train)
metrics['tuning_time(s)'].append(tuning_time)
rf_best_params

{'max_depth': 4,
 'max_features': 'sqrt',
 'n_estimators': 100,
 'random_state': 12345}

In [21]:
# Medir o tempo de treinamento apenas com os melhores parametros
rf_model = RandomForestRegressor(
    n_estimators = rf_best_params['n_estimators'],
    max_depth = rf_best_params['max_depth'],
    max_features = rf_best_params['max_features'],
    random_state = 12345
)
start_time = time.time()
rf_model.fit(X_enc_train, y_enc_train)
training_time = time.time() - start_time
metrics['training_time(s)'].append(training_time)

In [22]:
start_time = time.time()
rf_predictions = rf_model.predict(X_enc_test)
prediction_time = time.time() - start_time
metrics['prediction_time(s)'].append(prediction_time)

In [23]:
rf_rmse = math.sqrt(mean_squared_error(y_enc_test, rf_predictions))
metrics['RMSE'].append(rf_rmse)
print(f'REQM: {rf_rmse:.2f}')

REQM: 3674.58


### CatBoost

In [24]:
# Criar modelo e parametros para hipertuning
catboost_model = CatBoostRegressor()
cb_parameters = {
    'depth': [2, 4, 6],
    'learning_rate': [0.01, 0.05, 0.1],
    'iterations': [50, 100],
    'random_state': [12345],
    'verbose': [0]
}


tuning_time, cb_best_params = hyper_tuning(catboost_model, cb_parameters, X_gbm_train, y_gbm_train, categoric_features)

metrics['tuning_time(s)'].append(tuning_time)

cb_best_params

{'depth': 6,
 'iterations': 100,
 'learning_rate': 0.1,
 'random_state': 12345,
 'verbose': 0}

In [25]:
# Treinar apenas com os melhores parâmetros
catboost_model = CatBoostRegressor(
    depth=cb_best_params['depth'],
    learning_rate=cb_best_params['learning_rate'],
    iterations=cb_best_params['iterations'],
    random_state=12345,
    verbose=0
)

start_time = time.time()
catboost_model.fit(X_gbm_train, y_gbm_train, cat_features=list(categoric_features))
training_time = time.time() - start_time
metrics['training_time(s)'].append(training_time)

In [26]:
start_time = time.time()
catboost_predictions = catboost_model.predict(X_gbm_test)
prediction_time = time.time() - start_time
metrics['prediction_time(s)'].append(prediction_time)

In [27]:
catboost_rmse = mean_squared_error(y_gbm_test, catboost_predictions) ** 0.5
metrics['RMSE'].append(catboost_rmse)

### LightGBM

In [28]:
for c in categoric_features:
    X_gbm_train[c] = X_gbm_train[c].astype('category')
    X_gbm_test[c] = X_gbm_test[c].astype('category')

In [29]:
lgbm_model = lgb.LGBMRegressor()

In [30]:
# Descobrir os melhores parâmetros
lgbm_params = {
    'metric': ['rmse'],
    'max_depth': [2, 3, 4],
    'learning_rate': [0.05, 0.1],
    'random_state': [12345]
}

tuning_time, lgbm_best_params = hyper_tuning(lgbm_model, lgbm_params, X_gbm_train, y_gbm_train)

metrics['tuning_time(s)'].append(tuning_time)
lgbm_best_params

{'learning_rate': 0.1,
 'max_depth': 4,
 'metric': 'rmse',
 'objective': 'regression',
 'random_state': 12345}

In [31]:
# Treinar apenas com os melhores parâmetros
lgbm_model = lgb.LGBMRegressor(
    objective='regression',
    metric='rmse',
    max_depth=lgbm_best_params['max_depth'],
    learning_rate=lgbm_best_params['learning_rate'],
    random_state=12345
)

start_time = time.time()
lgbm_model.fit(X_gbm_train, y_gbm_train)
training_time = time.time() - start_time
metrics['training_time(s)'].append(training_time)

In [32]:
start_time = time.time()
lgbm_predictions = lgbm_model.predict(X_gbm_test)
prediction_time = time.time() - start_time
metrics['prediction_time(s)'].append(prediction_time)

In [33]:
lgbm_rmse = math.sqrt(mean_squared_error(y_gbm_test, lgbm_predictions))
metrics['RMSE'].append(lgbm_rmse)

## Análise do modelo

In [34]:
# Criando Dataframe com as métricas dos modelos testados
index = ['Linear Regression', 'Random Forest', 'CatBoost', 'LightGBM']
df_metrics = pd.DataFrame(metrics, index=index)
df_metrics.head()

,training_time(s),prediction_time(s),RMSE,tuning_time(s)
Linear Regression,15.155565,0.199157,2656.537980,N/A
Random Forest,7.504035,0.224466,3674.580766,103.31394
CatBoost,15.102199,0.124063,1734.250039,172.395295
LightGBM,2.900786,0.508779,1662.463731,24.956298


Ao analisar os dados percebemos que:<br>
1) LightGBM tem o menor tempo de treino, porém se considerarmos o tempo gasto com o tuning dos parâmetros, a Regressão Linear é mais rápida;<br>
2) O tempo de predição mais rápido é do CatBoost e o mais lento do LightGBM, porém é uma diferença que não tem impacto no nosso caso; <br>
3) O modelo que apresentou o menor Erro foi o LightGBM, seguido de perto pelo CatBoost e o pior modelo foi o de Floresta Aleatória; <br>

## Conclusões

Com base nas métricas que medimos, o melhor modelo a ser utilizado foi o LightGBM que possuio o segundo menor tempo de treino, pouco atrás da Regressão Linear, e a melhor qualidade das previsões, mesmo que tenha demorado mais tempo para fazer as previsões, esse tempo não foi considerável para alterar a decisão sobre o melhor modelo.

# Checklist

Digite 'x' para verificar. Em seguida, pressione Shift + Enter.

- [x]  O Jupyter Notebook está aberto
- [x]  O código está livre de erros
- [x]  As células com o código foram organizadas em ordem de execução
- [x]  Os dados foram baixados e preparados
- [x]  Os modelos foram treinados
- [x]  A análise de velocidade e qualidade dos modelos foi realizada